## 以命名實體集問題關聯性在知識庫尋找解答

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import time
import csv

#### 將知識庫轉為 csv 

In [2]:
file = './nlpcc-iccpol-2016.kbqa.kb'

kb = []

k = 0

ptm = time.time()

with open(file,'r', encoding="utf-8") as f:

    for line in f:
        
        kb.append(line.strip().split(' ||| '))
        
        if ( k + 1 ) % 10000000 == 0 :
            
            print( k + 1 )
        
        k += 1
        
time.time() - ptm

10000000
20000000
30000000
40000000


266.2538595199585

In [3]:
ptm = time.time()

with open('kb1.csv','w',newline ='', encoding='utf-8-sig') as f:
        
        mywrite = csv.writer(f)
        
        for i in range(len(kb)) :
            
            if i == 0 :
                
                mywrite.writerow(['entity','relation','answer'])
                
            else :
                
                mywrite.writerow(kb[i])
                
                
            if (i+1) % 5000000 == 0 :
                
                print( i + 1 )
        
        
time.time() - ptm

5000000
10000000
15000000
20000000
25000000
30000000
35000000
40000000


141.39819049835205

In [4]:
kbdf = pd.read_csv('kb.csv')
kbdf.head()

,entity,relation,answer
0,空气干燥,中文名,空气干燥
1,空气干燥,外文名,air drying
2,空气干燥,形式,两个
3,空气干燥,作用,将空气中的水份去除
4,罗育德,别名,罗育德


In [5]:
relation_test = pd.read_csv('04_Entity_testing_data.csv')
relation_test['purpose'] = relation_test['purpose'].apply(lambda x : eval(x))
uniq_kb_entity = list(set(kbdf['entity']))
uniq_kb_entity[0] = 'NNAANNA'
relation_test.head()

,question,entity,purpose
0,"['你', '知', '道', '计', '算', '机', '应', '用', '基', ...",计算机应用基础,"[你, 知, 道, 这, 本, 书, 的, 作, 者, 是, 谁, 吗, ？]"
1,"['计', '算', '机', '应', '用', '基', '础', '这', '本', ...",计算机应用基础,"[这, 本, 书, 的, 出, 版, 社, 是, 那, 个, ？]"
2,"['告', '诉', '我', '高', '等', '数', '学', '的', '出', ...",高等数学,"[告, 诉, 我, 的, 出, 版, 时, 间, 是, 什, 么, 时, 候, ？]"
3,"['我', '想', '知', '道', '戴', '维', '斯', '是', '什', ...",戴维斯,"[我, 想, 知, 道, 是, 什, 么, 国, 家, 的, 人, ？]"
4,"['你', '知', '道', '高', '等', '数', '学', '的', 'i', ...",高等数学,"[你, 知, 道, 的, i, s, b, n, 吗, ？]"


#### 找出命名實體後，以 testing 的 purpose 以字串相似度找出最高的知識庫 relation ，最後得出問題解答。

In [6]:
import operator

import re

a = dict(zip(kbdf['entity'], kbdf.index))

a = dict(sorted(a.items(), key=operator.itemgetter(1)))

b = list(a.values())

In [7]:
import re

import math

from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

not_in_kb_index = []

entity, relation, answer = [], [], []

ptm = time.time()

for i in range(len(relation_test)) :
      
    try:
        
        Entity = re.sub("《|》", "", relation_test['entity'][i])
               
        inin = b.index(a[Entity])
    
        if inin == 0 :

            start = 0

        else :

            start = b[b.index(a[Entity]) - 1] + 1
        
        df = kbdf.loc[start:a[Entity]]
               
        kk = df['relation'].apply(lambda x : similar(x, ''.join(relation_test['purpose'][i])))
        
        entity.append(Entity)
        
        relation.append(df['relation'][np.argmax(kk)])
        
        answer.append(df['answer'][np.argmax(kk)])
        
    except:
        
        not_in_kb_index.append(i)
        
        entity.append(Entity)
        
        relation.append('不知道')
        
        answer.append('不知道')
        
        pass
    
    if (i+1) % 200 == 0 : 
    
        print(i+1)
        
relation_test['entity'] = entity
relation_test['relation'] = relation
relation_test['answer'] = answer


time.time() - ptm

C:\Users\a0972\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:51: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800


506.27555203437805

In [8]:
relation_test.head()

,question,entity,purpose,relation,answer
0,"['你', '知', '道', '计', '算', '机', '应', '用', '基', ...",计算机应用基础,"[你, 知, 道, 这, 本, 书, 的, 作, 者, 是, 谁, 吗, ？]",作者,刘晓斌、魏智荣、刘庆生
1,"['计', '算', '机', '应', '用', '基', '础', '这', '本', ...",计算机应用基础,"[这, 本, 书, 的, 出, 版, 社, 是, 那, 个, ？]",出版社,化学工业出版社
2,"['告', '诉', '我', '高', '等', '数', '学', '的', '出', ...",高等数学,"[告, 诉, 我, 的, 出, 版, 时, 间, 是, 什, 么, 时, 候, ？]",出版时间,2014年3月
3,"['我', '想', '知', '道', '戴', '维', '斯', '是', '什', ...",戴维斯,"[我, 想, 知, 道, 是, 什, 么, 国, 家, 的, 人, ？]",国籍,荷兰
4,"['你', '知', '道', '高', '等', '数', '学', '的', 'i', ...",高等数学,"[你, 知, 道, 的, i, s, b, n, 吗, ？]",别名,高等数学


In [9]:
relation_test.to_csv('04_answer_prediction_data.csv', index = False, encoding='utf_8_sig')

In [10]:
output = open('./submit/04_Kbqa.testing-data',"w+", encoding="utf-8")

question_str = "<question"
answer_str = "<answer"
start_str = "============="

k = 0

j = 1

with open('04_Kbqa.testing-data','r', encoding="utf-8") as f:
    
    for line in f : 
        
        if question_str in line :
            
            output.write(line)
            
            output.write("<triple id=%s>"%(j) + '\t' + entity[k] + ' ||| ' + relation[k] + ' ||| ' + answer[k] + '\n')
        
            j += 1    
            
        if answer_str in line :
            
            output.write(line[:-1] + answer[k] + '\n')
            
            aa = line
            
            k += 1
            
        if start_str in line :
           
            output.write(line)
            
        